# How to download sequence files with UDN Gateway and FileService APIs
This example uses python but the concepts can be converted to any scripting language. 

## Setup
Import python packages and setup header information.  Two different headers are used in this example: one for the UDN Gateway API and the other for the FileService API. 

In [ ]:
import requests
import json

A authorization token is needed to access the UDN Gateway API.  This token is shown in the dictionary below as the `Authorization` token. Login to the web UDN Gateway and navigate to `/login/token/` to obtain an authorization token. 

A second token is needed to access the details about files stored in FileService which is the applicaiton that manages metadata for all UDN Gateway sequencing files.  This token is shown in the dictionary below as `FSAuthorization`.  The `FSAuthorization` key is specific to the UDN Gateway API.    

Login to FileService and navigate to `/filemaster/token/` to obtain an authorization token. 

development: https://fileservice-ci.dbmi.hms.harvard.edu/

production: https://fileservice.dbmi.hms.harvard.edu/



In [ ]:
gateway_token = 'xxxxx'
fileservice_token = 'xxxx'

In [ ]:
headers = {'Content-Type': 'application/json', 
           'Authorization': 'Token ' + gateway_token, 
           'FSAuthorization': 'FSToken ' + fileservice_token}

## Get patient indexed file metadata from the UDN Gateway
To get a list of files associated with a specific patient make a GET request to  

`/api/sequence/files/<patient_udnid>/`

where `<patient_udnid>` is the UDN ID of the patient and can be obtained either through the web UDN Gateway interface or using the UDN Gateway API.  

This request returns a list of JSON objects that provide details for each file associated with a patient.  

In [ ]:
gateway_system_url = 'https://gateway.undiagnosed.hms.harvard.edu/api'

In [ ]:
patient_udnid = 'UDN799247'

-----
Data is returned as a list of JSON objects.  This example looks at the first object in that list as indicated by the `[0]` in the last line of the following code block.  The FileService UUID is required to obtain a secure download link.  The filename is optional but recommended.  

----

In [ ]:
url = gateway_system_url + '/sequence/files/'+ patient_udnid +'/'
r = requests.get(url, headers=headers)
print r
r.json()[0]

In [ ]:
fileserviceuuid = r.json()[0]['fileserviceuuid']

In [ ]:
filename = r.json()[0]['filename']

## Get file download URL from FileService
First setup a new set of header information.  The `Token` value here is the same as the `FSToken` in the header used in the header for the UDN Gateway API.  This is now accessing the FileService API.  

In [ ]:
fs_headers = {'Content-Type': 'application/json; charset=UTF-8', 
              'Authorization': 'Token '+ fileservice_token
             }

Use the `fileserviceuuid` from the file metadata returned in the previous section.  Then make a GET request to the following url endpoint to obtain a signed download url from FileService. 

`/filemaster/api/file/<file_uuid>/download/`

In [ ]:
fileservice_system_url = 'https://fileservice.dbmi.hms.harvard.edu/filemaster/api'

In [ ]:
url = fileservice_system_url + '/file/' + fileserviceuuid + '/download/'
r = requests.get(url, headers=fs_headers)
print r
r.json()

In [ ]:
download_url = r.json()['url']

### File Metadata

A GET request to the following url endpoint returns the full set of metadata associated with the file

```/filemaster/api/file/<file_uuid>/```

In [ ]:
url = fileservice_system_url + '/file/' + fileserviceuuid + '/'
r = requests.get(url, headers=fs_headers)
print r
r.json()

# Download file
Then use a download tool like wget to download the file from the `url` field that is returned. Be sure to include the quotation marks.   

```
wget -O "<filename>" "<url>"
```

The example below shows how to call `wget` programmatically within a python script

In [ ]:
from subprocess import call

In [ ]:
call('wget -O "'+filename+'" "'+download_url+'"', shell=True)